In [23]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib


In [24]:
# Load the saved models
# Load sklearn models with joblib
lgbm =joblib.load('C:/Users/BRENDA MUTAI/Documents/crypto-classifier/models/lightgbm_model.pkl')
xgb=joblib.load('C:/Users/BRENDA MUTAI/Documents/crypto-classifier/models/xgboost_model.pkl')
cat=joblib.load('C:/Users/BRENDA MUTAI/Documents/crypto-classifier/models/catboost_model.pkl') 

# Load Keras LSTM model with keras
lstm = load_model('C:/Users/BRENDA MUTAI/Documents/crypto-classifier/models/lstm_model.h5')


# Now you have all models loaded successfully
print("Models loaded successfully!")
print(f"LightGBM: {type(lgbm)}")
print(f"XGBoost: {type(xgb)}")
print(f"CatBoost: {type(cat)}")
print(f"LSTM: {type(lstm)}")

Models loaded successfully!
LightGBM: <class 'lightgbm.sklearn.LGBMClassifier'>
XGBoost: <class 'xgboost.sklearn.XGBClassifier'>
CatBoost: <class 'catboost.core.CatBoostClassifier'>
LSTM: <class 'keras.src.models.sequential.Sequential'>


In [19]:
models = {
    "LightGBM": lgbm,
    "XGBoost": xgb,
    "CatBoost": cat,
    "LSTM": lstm
}
def evaluate_model(name, model, X_test, y_test):
    print(f"\n===== {name} Evaluation =====")
    
    # ---- PREDICTION ----
    
    if name == "LSTM":
        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)   # for classification
    else:
        y_pred = model.predict(X_test)
    
    # ---- METRICS ----
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    precision_buy = precision_score(y_test, y_pred, labels=[1], average="macro")
    recall_buy = recall_score(y_test, y_pred, labels=[1], average="macro")
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Macro F1-score: {f1:.4f}")
    print(f"BUY Precision (class=1): {precision_buy:.4f}")
    print(f"BUY Recall (class=1): {recall_buy:.4f}")
    
    # ---- CONFUSION MATRIX ----
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()
